In [ ]:
import numpy as np 
import pandas as pd
import pandas_profiling as pp
import seaborn as sns
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, RandomForestRegressor

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Import Data

In [ ]:
train = pd.read_csv("../input/learn-together/train.csv", index_col='Id')
test = pd.read_csv("../input/learn-together/test.csv",index_col='Id')


Imputing Hillshade_3pm zeros as per https://www.kaggle.com/arateris/stacked-classifiers-for-forest-cover and also checking the correlations with Hillshade_9am and Hillshade_3pm to find the features need to impute. 

In [ ]:
# questionable_0 = ['Hillshade_9am', 'Hillshade_3pm']
# all_data = train.append(test)
# for col in questionable_0:
#     all_data_0 = all_data[all_data[col] == 0].copy()
#     all_data_non0 = all_data[all_data[col] != 0].copy()
#     corr = all_data_non0.corr()[col]
#     corr = np.abs(corr[corr.index != col]).sort_values(ascending = False)
    
#     sns.set_style('whitegrid')
#     plt.figure(figsize = (17,4))
#     fig = sns.barplot(x=corr.index, y=corr)
#     fig.set_xticklabels(labels = corr.index, rotation=90)
#     plt.ylabel('Correlation')
#     plt.title(col)
#     plt.show()

As we see Aspect, Slope, Hillshade_9am, Hillshade_Noon are higly correlated with Hillshade_3pm. Going to impute this variable alone for this iteration,Used https://www.kaggle.com/hoangnguyen719/part-1-eda-and-feature-engineering to plot these correlation graphs. Will consider imputing Hillshade_9am variable in the next iteration

In [ ]:
# corr = train.corr()['Slope']
# corr.style.background_gradient(cmap='coolwarm')

#replacing the zeros for better guess, mainly to avoid zeros in the feature engineering and fake outliers. 
num_train=len(train)
all_data = train.append(test)
cols_for_HS = ['Aspect','Slope', 'Hillshade_9am','Hillshade_Noon']
HS_zero = all_data[all_data.Hillshade_3pm==0]
HS_zero.shape

HS_train = all_data[all_data.Hillshade_3pm!=0]
# res = cross_val_score(RandomForestRegressor(n_estimators=100), HS_train.drop('Hillshade_3pm',axis=1), HS_train.Hillshade_3pm, n_jobs=-1, verbose=True)
# print(res) #[0.9996774  0.99989463 0.9999186 ]
##actually, the CV is so close to zero there is actually no new information here..keeping it for simplicity
rf_hs = RandomForestRegressor(n_estimators=100).fit(HS_train[cols_for_HS], HS_train.Hillshade_3pm)
out = rf_hs.predict(HS_zero[cols_for_HS]).astype(int)
all_data.loc[HS_zero.index,'Hillshade_3pm'] = out
train= all_data[:num_train]
test= all_data[num_train:]

Reused the features https://www.kaggle.com/jakelj/basic-ensemble-model

In [ ]:
train['EV_DTH'] = (train.Elevation - train.Vertical_Distance_To_Hydrology)
test['EV_DTH'] = (test.Elevation - test.Vertical_Distance_To_Hydrology)


train['EH_DTH'] = (train.Elevation -  (train.Horizontal_Distance_To_Hydrology *0.2))
test['EH_DTH'] = (test.Elevation -  (test.Horizontal_Distance_To_Hydrology *0.2))

train['Dis_To_Hy'] = (((train.Horizontal_Distance_To_Hydrology **2) + (train.Vertical_Distance_To_Hydrology **2))**0.5)
test['Dis_To_Hy'] = (((test.Horizontal_Distance_To_Hydrology **2) + (test.Vertical_Distance_To_Hydrology **2))**0.5)

train['HyF_1'] = (train.Horizontal_Distance_To_Hydrology + train.Horizontal_Distance_To_Fire_Points)
test['HyF_1'] = (test.Horizontal_Distance_To_Hydrology + test.Horizontal_Distance_To_Fire_Points)

train['HyF_2'] = (train.Horizontal_Distance_To_Hydrology - train.Horizontal_Distance_To_Fire_Points)
test['HyF_2'] = (test.Horizontal_Distance_To_Hydrology - test.Horizontal_Distance_To_Fire_Points)

train['HyR_1'] = (train.Horizontal_Distance_To_Hydrology + train.Horizontal_Distance_To_Roadways)
test['HyR_1'] = (test.Horizontal_Distance_To_Hydrology + test.Horizontal_Distance_To_Roadways)

train['HyR_2'] = (train.Horizontal_Distance_To_Hydrology - train.Horizontal_Distance_To_Roadways)
test['HyR_2'] = (test.Horizontal_Distance_To_Hydrology - test.Horizontal_Distance_To_Roadways)


train['FiR_1'] = (train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Roadways)
test['FiR_1'] = (test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Roadways)

train['FiR_1'] = (train.Horizontal_Distance_To_Fire_Points - train.Horizontal_Distance_To_Roadways)
test['FiR_1'] = (test.Horizontal_Distance_To_Fire_Points - test.Horizontal_Distance_To_Roadways)

train['Avg_shade'] = ((train.Hillshade_9am + train.Hillshade_Noon + train.Hillshade_3pm) /3)
test['Avg_shade'] = ((test.Hillshade_9am + test.Hillshade_Noon + test.Hillshade_3pm) /3)

train['Morn_noon_int'] = ((train.Hillshade_9am + train.Hillshade_Noon) / 2)
test['Morn_noon_int'] = ((test.Hillshade_9am + test.Hillshade_Noon) / 2)

train['noon_eve_int'] = ((train.Hillshade_3pm + train.Hillshade_Noon) / 2)
test['noon_eve_int'] = ((test.Hillshade_3pm + test.Hillshade_Noon) / 2)

train['Slope2'] = np.sqrt(train.Horizontal_Distance_To_Hydrology**2 + train.Vertical_Distance_To_Hydrology**2)
test['Slope2'] = np.sqrt(test.Horizontal_Distance_To_Hydrology**2 + test.Vertical_Distance_To_Hydrology**2)

Testing features from this notebook https://www.kaggle.com/justfor/ensembling-and-stacking-with-heamy in search for a better defined domain knowledge

In [ ]:
test.columns

In [ ]:
# train = add_feats(train)    
# test = add_feats(test)  
# covertype_labelcolumn = train['Cover_Type']
# train.drop(['Soil_Type15', 'Soil_Type7','Cover_Type'], axis=1, inplace = True)
label = train['Cover_Type']
train.drop(['Soil_Type15', 'Soil_Type7','Cover_Type'], axis=1, inplace = True)
test.drop(['Soil_Type15', 'Soil_Type7','Cover_Type'], axis=1, inplace = True)
all_data=train.append(test)


In [ ]:
# questionable_0 = ['Hillshade_9am', 'Hillshade_3pm']

# for col in questionable_0:
#     all_data_0 = all_data[all_data[col] == 0].copy()
#     all_data_non0 = all_data[all_data[col] != 0].copy()
#     corr = all_data_non0.corr()[col]
#     corr = np.abs(corr[corr.index != col]).sort_values(ascending = False)
    
#     sns.set_style('whitegrid')
#     plt.figure(figsize = (17,4))
#     fig = sns.barplot(x=corr.index, y=corr)
#     fig.set_xticklabels(labels = corr.index, rotation=90)
#     plt.ylabel('Correlation')
#     plt.title(col)
#     plt.show()

In [ ]:
# n_train=len(train.drop(['Cover_Type'], axis =1))
# print (n_train)
n_train = len(test.index)
print (n_train)

Adding PCA components from the entire raw features . testing the PCA code from https://www.kaggle.com/edumunozsala/feature-eng-and-a-simple-stacked-model to see if the PCA increase the performance of the algorithm. Also ignore Gaussian Mixture modeling in this run

In [ ]:
# from sklearn.decomposition import PCA
    
# #Apply PCA to obtain new features based in all datasets (train and test)
# def add_PCA_features(X):
#     pca = PCA(n_components=0.99).fit(X)
#     X_pca = pca.transform(X)
    
#     return X_pca

# components = add_PCA_features(all_data)
# # components = add_PCA_features(train)
# # print (train.shape)
# # print (test.shape)
# # print (all_data.shape)
# # print (n_train)


# print('PCA components dimension: ',components.shape)
# # print (components)
# for i in range(components.shape[1]):
#     col_name= 'pca'+str(i+1)
#     test[col_name] = components[:n_train, i]
#     train[col_name] = components[n_train:, i]

# print(test.shape,train.shape)

In [ ]:
train.columns

Test data being so large compared to Train, using GaussianMixture Modeling fit on test data and using the data categorized into clusters as an added feature to train and test data sets. Reused from https://www.kaggle.com/stevegreenau/stacking-multiple-classifiers-clustering

In [ ]:
# from sklearn.mixture import GaussianMixture

# gmix = GaussianMixture(n_components=10)
# gmix.fit(test)

# train['Test_Cluster'] = gmix.predict(train.drop(['Cover_Type'], axis =1))
# test['Test_Cluster'] = gmix.predict(test)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, label, test_size=0.2, random_state = 2)
# X_train, X_val, y_train, y_val = train_test_split(train.drop(['Cover_Type'], axis =1), train['Cover_Type'], test_size=0.2, random_state = 2)

Using L1 regularization to select the most telling features . on Training data and keep only the features selected by L1 regularization. Also drop unselected columns from validation and test datasets

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectFromModel

# X, y = X_train, y_train

# """ Return selected features using logistic regression with an L1 penalty """
# logistic = LogisticRegression(C=0.1, penalty="l1", random_state=7).fit(X, y)
# model = SelectFromModel(logistic, prefit=True)
# X_new = model.transform(X)
# selected_features = pd.DataFrame(model.inverse_transform(X_new), 
#                              index=X.index,
#                              columns=X.columns)

# selected_columns = selected_features.columns[selected_features.var() != 0]


In [ ]:
# len(X_train.columns)


In [ ]:
# len(selected_columns)

In [ ]:
# X_train = X_train[selected_columns]
# X_val = X_val[selected_columns]
# len(X_train.columns)

In [ ]:
# len(test.columns)
# # test = test[selected_columns]

In [ ]:
train.nunique()
train.columns

Let's use pandas profiling to get a report on EDA views

Let's delete the Id column in the training set but store it for the test set before deleting

# Model Training

Let's use 80% of the Data for training, and 20% for validation. We'll then train a simple Random Forest Classifier with 100 trees

In [ ]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, ExtraTreesClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import cross_val_score,cross_validate, train_test_split, GridSearchCV, StratifiedKFold
from mlxtend.classifier import StackingCVClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(train.drop(['Cover_Type'], axis=1), train['Cover_Type'], test_size=0.2, random_state = 42)
# X_train, X_val, y_train, y_val = train_test_split(train.drop(['Cover_Type'], axis =1), train['Cover_Type'], test_size=0.2, random_state = 25)
# random_state = 1
# X_train, X_val, y_train, y_val = train_test_split(train,covertype_labelcolumn, test_size=0.2, random_state = random_state)
# X_train, X_val, y_train, y_val = train_test_split(train.drop(['Cover_Type'], axis =1), train['Cover_Type'], test_size=0.2, random_state = 2)


Has been using boosting all this while, testing stacking the classifiers on the same features,  credits to https://www.kaggle.com/kwabenantim/forest-cover-stacking-multiple-classifiers

In [ ]:
# X_train = train.drop(['Cover_Type'], axis =1)
# y_train = train['Cover_Type']

print('> Setting up classifiers')
random_state = 1
max_features = min(30, X_train.columns.size)


ab_clf = AdaBoostClassifier(n_estimators=200,
                            base_estimator=DecisionTreeClassifier(
                                min_samples_leaf=2,
                                random_state=random_state),
                            random_state=random_state)

et_clf = ExtraTreesClassifier(n_estimators=300,
                              min_samples_leaf=2,
                              min_samples_split=2,
                              max_depth=50,
                              max_features=max_features,
                              random_state=random_state,
                              n_jobs=1)

lg_clf = LGBMClassifier(n_estimators=300,
                        num_leaves=128,
                        verbose=-1,
                        random_state=random_state,
                        n_jobs=1)

rf_clf = RandomForestClassifier(n_estimators=300,
                                random_state=random_state,
                                n_jobs=1)

ensemble = [('AdaBoostClassifier', ab_clf),
            ('ExtraTreesClassifier', et_clf),
            ('LGBMClassifier', lg_clf),
            ('RandomForestClassifier', rf_clf)]


print('> Cross-validating classifiers')
for label, clf in ensemble:
    score = cross_val_score(clf, X_train, y_train,
                            cv=5,
                            scoring='accuracy',
                            verbose=0,
                            n_jobs=-1)

    print('  -- {: <24} : {:.3f} : {}'.format(label, np.mean(score), np.around(score, 3)))


print('> Fitting stack')
# - https://www.kaggle.com/itslek/stack-blend-lrs-xgb-lgb-house-prices-k-v17
# - https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
# - https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python

stack = StackingCVClassifier(classifiers=[ab_clf, et_clf, lg_clf, rf_clf],
                             meta_classifier=rf_clf,
                             cv=5,
                             stratify=True,
                             shuffle=True,
                             use_probas=True,
                             use_features_in_secondary=True,
                             verbose=1,
                             random_state=random_state,
                             n_jobs=-1)

In [ ]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
# low_cardinality_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
# #                         X_train[cname].dtype == "object"]
# low_cardinality_cols = [cname for cname in X_train.columns if X_train[cname].nunique() == 2 ]

In [ ]:
# model = RandomForestClassifier(n_estimators=1000, max_depth=10)
# model = XGBClassifier(n_estimators=500, learning_rate=0.1, max_depth=5)
# model = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=10)
# model = XGBClassifier()


In [ ]:
# param = {'n_estimators': [100, 500,1000],
#          'learning_rate': [0.1, 0.05],
#          'max_depth': [3, 4, 5, 6]}
# grider = GridSearchCV(model, param, n_jobs=-1, cv=5, scoring='accuracy', verbose=True)

In [ ]:
stack.fit(X_train, y_train)
# model.fit(X_train, y_train)
# grider.fit(X_train, y_train)

In [ ]:
# print (grider.best_params_)
# print(grider.best_score_)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, mean_absolute_error

In [ ]:
# train_pred = model.predict(X_train)
stack.score(X_train, y_train)

In [ ]:
# model.score(X_train, y_train)
# train_pred = model.predict(X_train)
# mae_1 = mean_absolute_error(train_pred, y_train) # Your code here

# # Uncomment to print MAE
# print("Mean Absolute Error:" , str(mae_1))

In [ ]:
# model = XGBClassifier(n_estimators=100)
# model.fit(X_train, y_train)
# # make predictions for test data
# y_pred = model.predict(X_val)
# predictions = [round(value) for value in y_pred]
# # evaluate predictions
# accuracy = accuracy_score(y_val, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
predictions = stack.predict(X_val)
# # mean_absolute_error(predictions, y_val)
accuracy_score(y_val, predictions)

Model has a 100% accuracy on the training set and ~ 89.947% on the test set. XGboost classifier instead of randomforest and some feature engineering (added a few features inspired from earlier work)

Added PCA components in addition to the features, training accuracy 99.99917% and 89.087301% on test set.

Removed PCA but imputing Hillshade_3pm "0" values using Randomforestregressior as mentioned in https://www.kaggle.com/arateris/stacked-classifiers-for-forest-cover , training accuracy 100% and 90.244% on test set.

# Final Predictions

In [ ]:
# test.head()

print('Missing test data? ', test.isnull().any().any())
# print(test.columns[test.isnull().any()])

In [ ]:
test_pred = stack.predict(test)

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'ID': test.index,
                       'Cover_Type': test_pred})
output.to_csv('submission.csv', index=False)

Planned next steps

* Feature Extraction/ Feature engineering
* Introduce Regularization techniques
* Hyper parameter tuning - Grid-Search to find the optimal parameters for our classifier 
* Xgboost classifier

Stacking multiple classifiers on the same features provided a test accuracy of 89.35% and train accuracy of 99%